In [26]:
# Ensure you have the following installed:
# numpy
# os
# compas
# compas_fd
# warnings
# traceback
# pickle

# Install by running:
# !pip install -r requirements.txt

In [27]:
import numpy as np
import pickle

# G-code generation codes are cloned from https://github.com/tibor-barsi/GcodeGenerator. Tibor Barsi is the author of the code, and was a PhD student at the Ladisk lab of the University of Ljubljana
#  We should be careful with crediting the author of the code, if we ever want to make these code public.
from src.g_code_generation_copy.gcode_generator import G_code_generator
from src.g_code_generation_copy.tool_changer_functions import save_params, load_params, printer_start, load_tool, unload_tool, tool_change, take_photo, play_sound, printer_stop
# from src.additional_functions import *
from src.network import Network_custom, replace_brackets
import os

BYU_UW_root = r"G:\.shortcut-targets-by-id\1k1B8zPb3T8H7y6x0irFZnzzmfQPHMRPx\Illimited Lab Projects\Research Projects\Spiders\BYU-UW"

### Step 1: Define the network

In [28]:
# Note that the coordinates of the vertices will change after equilibrium is reached.
model_name  = 'Validation_structure_0'
vertices    = np.array([[-10, 0, 0], [-5, 0,0], [0, 0, 0], [5, 0, 0], [10, 0, 0], [0,-10,0], [0,-5,0], [0,5,0], [0,10,0], [-2.5, 2.5, 0]])*10  # The coordinates of the vertices in mm
edges       = [[0, 1], [1, 2], [2, 3], [3, 4], [5, 6], [6, 2], [2, 7], [7, 8], [1, 6], [6, 3], [3, 7], [7,9], [9,1]]                  # The edges of the network.
paths       = [[0, 1, 2, 3], [4, 5, 6,7], [12, 8, 9, 10, 11]]   # 3D printable paths. Ensure an edge is not flipped.
directions  = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]          # Flip the curvature of an edge by setting the direction to -1.

q_ring      = .5
q_connectors = .5
q_center    = .1
q           = np.array([q_connectors, q_center, q_center, 3*q_connectors, q_connectors, q_center, q_center, q_connectors, 5*q_ring, q_ring, q_ring, q_ring*2, q_ring*2])/20
fixed       = [0, 4, 5, 8]

# Create the network
net = Network_custom.from_fd(vertices, edges, q, fixed, paths = paths, dir = directions)

# The topology of structure is fixed after generation, but you can update force densities after making the network
# q[0] = .10
# net.update_shape(q)

# Plot the network like this
net.net_plot(color=True, elables = False, vlabels = True)

(net.vertices == vertices).all() # Please understand that this is False

True

Here is the same network but elements 11 and 12 are combined into a single element.

In [29]:
# Note that the coordinates of the vertices will change after equilibrium is reached.
model_name  = 'Validation_structure_1'
vertices    = np.array([[-10, 0, 0], [-5, 0,0], [0, 0, 0], [5, 0, 0], [10, 0, 0], [0,-10,0], [0,-5,0], [0,5,0], [0,10,0]])*10  # The coordinates of the vertices in mm
edges       = [[0, 1], [1, 2], [2, 3], [3, 4], [5, 6], [6, 2], [2, 7], [7, 8], [1, 6], [6, 3], [3, 7], [7,1]]                  # The edges of the network.
paths       = [[0, 1, 2, 3], [4, 5, 6,7], [11, 8, 9, 10]]   # 3D printable paths. Ensure an edge is not flipped.
directions  = [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]          # Flip the curvature of an edge by setting the direction to -1.

q_ring      = .5
q_connectors = .5
q_center    = .1
q           = np.array([q_connectors, q_center, q_center, q_connectors, q_connectors, q_center, q_center, q_connectors, q_ring, q_ring, q_ring, q_ring])/20
fixed       = [0, 4, 5, 8]

# Create the network
net = Network_custom.from_fd(vertices, edges, q, fixed, paths = paths, dir = directions)

# The topology of structure is fixed after generation, but you can update force densities after making the network
# q[0] = .10
# net.update_shape(q)

# Plot the network like this
net.net_plot(color=True, elables = True, vlabels = False)

(net.vertices == vertices).all() # Please understand that this is False

False

### Step 2: Materialize

In [30]:
TPU = {'E':130, 'v':0.3897, 'p':1.18e-9, 'A':0.11*0.71, 'name': 'TPU conductive'} # Conductive TPU. Manufacturer Ninjatek Eel

net.set_material(TPU) # By saving the material properties in the network, we can easily see how the network was constructed.

# If all elements will get the same material, you can use the following line
E = [TPU['E']]*len(net.edges)
A = [TPU['A']]*len(net.edges)
l0, l_scalar = net.materialize(E, A)

# Yielding the initial lengths of each element. And the following scalar: min(l0/l1)
net.l0, net.l_scalar

(array([58.79661802, 30.77636589, 30.77636589, 58.79661802, 58.79661802,
        30.77636589, 30.77636589, 58.79661802, 39.85692416, 39.85692416,
        39.85692416, 39.85692416]),
 0.8552235348621727)

### Step 3: Define elements as a curves

In [31]:
# Scale down the web with network.l_scalar
reference_point = [0,0,0]                         # The network will be scaler relative to this point
net.scale_vertices(reference_point, net.l_scalar) # If you don't provide a scalar, it will use network.l_scalar automatically

#  You can plot the scaled network like this
# net.net_plot(color=True, plot_type='scaled')

# Determine the Radius and Angle of the circle that define the arc of each element
R, th = net.arc_param()
# for the arc length (net.l1 * net.l_scalar) is used unless specified otherwise, for the cord length (net.l0) is used unless specified otherwise
xyz = net.arc_points(n = 100) 

net.net_plot(color=True, plot_type='arcs', elables=True)

Flipping edge 2, 6, and 12 would make the network smoother

In [32]:
# net.flip_curve(2, n = 100)          # Flip the curvature of one edge
net.flip_curves([2, 6], n = 100)       # Flip the curvature of multiple edges

net.net_plot(color=True, plot_type='arcs')

### Step 4: Account for intersections. 
Curved paths and intersections are automatically set up in the backend

In [33]:
printing_params = load_params(r'DATA/NT_Eel_0.2mm_og.json')
# When no interpolation function is provided, linear interpolation is used. No other interpolation function is implemented (yet?).
net.jump_at_intersection(intersection_width = 2, intersection_height = 2, interpolation_function=None) # These settings are just for visibility, this would be way to much
# These setting are more appropriate for printing
# net.jump_at_intersection(intersection_width = printing_params['d_nozzle']*1.5, intersection_height = printing_params['layer_height'], interpolation_function=None) 
net.net_plot(color=True, plot_type='arcs', elables=True)

Add loops to easily attach the printed network to a frame

In [34]:
alpha_loop = np.deg2rad(6) # The angle of the loop
L_loop  = 11                # The length of the loop
n_points = 60               # The number of points in the loop

start_loop_bools = [True, True, False]
end_loop_bools = [True, True, False]

net.all_loop_to_path(start_loop_bools, end_loop_bools, L_loop, alpha_loop, n_points)

Sometimes it takes a while before material starts extruding. This should preferably be fixed in the printer set up, but can me forced by adding a running start to a path

In [35]:
start_loop_bools = [False, False, False]
end_loop_bools = [False, False, True]
L_loop  = 2
n_points = 5
net.add_running_start(start_loop_bools, end_loop_bools, L_loop, n_points)

In [36]:
net.save_network(os.path.join(BYU_UW_root, 'networks', model_name + '_net.pkl'))

### Step 5: Generate G-code
Import and finalize printing parameters for materials and starting and end .gcode for the top. The start gcode has additional settings for temperatures and Linear Advance (K-factor). Linear advance is important for TPU as it is very visco-elastic. Linear advance can account for this

In [37]:
printing_params = load_params(r'DATA/NT_Eel_0.2mm_og.json')
start_gcode     = open(r'DATA/start_gcode.gcode', 'r').read()
end_gcode       = open(r'DATA/end_gcode.gcode', 'r').read()

temperature_settings = {'first_layer_bed_temperature': 65, 'first_layer_temperature':205, 'K-factor': 0.20}
g_code = replace_brackets(start_gcode, temperature_settings)

comment = ''

bed_width = 250
bed_height = 210

gen = G_code_generator(printing_params=printing_params)

Make a Purge Line

In [38]:
point0, point1 = [20,0,0.3], [90,0,0.3]
g_code += gen.move_to_point(point0[0:2], point0[2] + printing_params['nozzle_lift'], comment='Move to start point')
g_code += gen.move_to_point(point0[0:2], point0[2], comment='Lower Nozzle')
g_code += gen.unretract()
g_code += gen._print_line(
        point0=point0,
        point1=point1,
        move_to_start=False, # move to start point without extruding
        extrude_factor=printing_params['extrude_factor']*2,
        comment=comment)
g_code += gen.retract()
g_code += gen.wipe(np.pi) # Wipe the nozzle horizontally

Make the paths

In [39]:
for path_i, cor_list in enumerate(net.paths_xyz):
    g_code += '\n;Path (' + str(path_i) + ' ' + str(path_i) + ')\n'
    cor_list   = np.array(cor_list)
    # Move the coordinates to the center of the bed and add the layer height
    cor_list[:,0] += bed_width/2
    cor_list[:,1] += bed_height/2 - 1
    cor_list[:,2] += printing_params['layer_height']
    g_code += '\n'
    # Move the coordinates to the start point
    g_code += gen.move_to_point(cor_list[0][0:2], cor_list[0][2] + printing_params['nozzle_lift'], comment='Move to start point')
    # Lower the nozzle
    g_code += gen.move_to_point(cor_list[0][0:2], cor_list[0][2], comment='Lower Nozzle')
    # Unretract the filament
    g_code += gen.unretract()
    # Print the path
    for point0, point1 in zip(cor_list[:-1], cor_list[1:]):
        g_code += gen._print_line(
            point0=point0,
            point1=point1,
            move_to_start=False, # move to start point without extruding
            extrude_factor = printing_params['extrude_factor'],
            comment=comment)
    # Retract the filament
    g_code += gen.retract()
    # Wipe the nozzle
    g_code += gen.wipe_from_last_points(g_code)    
    # Raise the nozzle
    g_code += gen.move_to_point(point1[0:2], point1[2] + printing_params['nozzle_lift'], speed_factor=0.5, comment='Raise Nozzle')

g_code += end_gcode
with open(os.path.join('DATA', 'generated_gcodes', model_name + '.gcode'), "w") as g_code_file:
    g_code_file.write(g_code)
print('G-code generated')

G-code generated
